In [278]:
import pandas as pd
import numpy as np

In [279]:
homicidios = pd.read_csv("data/censo_2000_pe/agressoes_arma_de_fogo_2000.csv")
homicidios = homicidios.rename(index=str, columns={"2000": "ano_2000"})
homicidios = homicidios.replace('-', np.nan)
homicidios['ano_2000'] = homicidios.ano_2000.astype("float64")
#homicidios

In [280]:
populacao = pd.read_csv("data/estimativa_populacao/populacao_pe_municipios.csv")
#populacao

In [281]:
microrregioes = pd.read_csv("data/microrregioes/municipios_microrregioes_pernambuco.csv")
#microrregioes

In [282]:
aux2 = pd.read_csv("data/censo_2000_pe/csv/edu_alunos_por_serie.csv")
#aux2.head()
codigo = aux2[['municipio', 'codigo_unidade_geografica']]
codigo = codigo.copy()
codigo['codigo_unidade_geografica'] = codigo['codigo_unidade_geografica'].apply(lambda x: int(x/10))
#codigo

In [283]:
codigo['municipio'] = codigo['municipio'].str.normalize('NFKD').\
                                                            str.encode('ascii', errors='ignore').str.decode('utf-8')
microrregioes['municipio'] = microrregioes['municipio'].str.normalize('NFKD').\
                                                            str.encode('ascii', errors='ignore').str.decode('utf-8')
microrregioes['microrregiao'] = microrregioes['microrregiao'].str.normalize('NFKD').\
                                                            str.encode('ascii', errors='ignore').str.decode('utf-8')
populacao['municipio'] = populacao['municipio'].str.normalize('NFKD').\
                                                            str.encode('ascii', errors='ignore').str.decode('utf-8')


In [284]:
tb_parcial = pd.concat([codigo.set_index("municipio"),\
                                microrregioes.set_index("municipio"), populacao.set_index("municipio")], axis= 1)
tb_parcial = tb_parcial.reset_index()
tb_parcial = tb_parcial.rename(columns={"index": "municipio"})

In [285]:
tb_parcial.head()


,municipio,codigo_unidade_geografica,microrregiao,pop
0,Abreu e Lima,260005,Recife,83486
1,Afogados da Ingazeira,260010,Vale do Pajeu,34524
2,Afranio,260020,Petrolina,14227
3,Agrestina,260030,Brejo,18713
4,Agua Preta,260040,Mata Meridional,25353


In [286]:
def filtro(codigo):
    aux=[]
    if (codigo >= 260001 and codigo <= 269999):
        aux.append(int(codigo))
    df_newcode = pd.DataFrame(aux, columns=["codigo_un"])
    return df_newcode

In [287]:
n_homicidios = pd.concat(map(lambda a: filtro(a.codigo_unidade_geografica).\
                             assign(homicidio = a.ano_2000), homicidios.itertuples()), ignore_index=True)
#n_homicidios.head()

In [288]:
tabela_ref = pd.concat([tb_parcial.set_index("codigo_unidade_geografica"),\
                                n_homicidios.set_index("codigo_un")], axis= 1)

In [289]:
tabela_ref.head()

,municipio,microrregiao,pop,homicidio
260005,Abreu e Lima,Recife,83486,32.0
260010,Afogados da Ingazeira,Vale do Pajeu,34524,8.0
260020,Afranio,Petrolina,14227,1.0
260030,Agrestina,Brejo,18713,8.0
260040,Agua Preta,Mata Meridional,25353,21.0


In [290]:
tabela_ref = tabela_ref.reset_index()
tabela_ref = tabela_ref.rename(columns={"index": "codigo_un"})
tabela_ref.head()

,codigo_un,municipio,microrregiao,pop,homicidio
0,260005,Abreu e Lima,Recife,83486,32.0
1,260010,Afogados da Ingazeira,Vale do Pajeu,34524,8.0
2,260020,Afranio,Petrolina,14227,1.0
3,260030,Agrestina,Brejo,18713,8.0
4,260040,Agua Preta,Mata Meridional,25353,21.0


In [291]:
def taxa(populacao, homicidios):
    aux=[]
    taxa_h = 100000*homicidios/populacao
    aux.append(taxa_h)
    df_taxa = pd.DataFrame(aux, columns=["taxa_homicidio"])
    return df_taxa

In [292]:
taxa = pd.concat(map(lambda a: taxa(a.pop, a.homicidio),\
                       tabela_ref.itertuples()), ignore_index=True)

In [293]:
tabela = pd.concat([tabela_ref, taxa], axis=1)
tabela.head()

,codigo_un,municipio,microrregiao,pop,homicidio,taxa_homicidio
0,260005,Abreu e Lima,Recife,83486,32.0,38.329780
1,260010,Afogados da Ingazeira,Vale do Pajeu,34524,8.0,23.172286
2,260020,Afranio,Petrolina,14227,1.0,7.028889
3,260030,Agrestina,Brejo,18713,8.0,42.751029
4,260040,Agua Preta,Mata Meridional,25353,21.0,82.830434


In [294]:
#tabela.to_csv('../2017-2-projeto-lbs4-eebls-ktcn/tabela_final.csv', index=False)
#tabela = pd.read_csv("../2017-2-projeto-lbs4-eebls-ktcn/tabela_final.csv")

In [295]:
def retirar_acento(dataframe):
    #dataframe['municipio'] = dataframe['municipio'].str.normalize('NFKD').\
                                                        #str.encode('ascii', errors='ignore').str.decode('utf-8')
    dataframe = dataframe.set_index('municipio')
    return dataframe

In [311]:
data_stack=[]

In [312]:
data = pd.read_csv("data/censo_2000_pe/educacao_pe/csv/pessoas_de_10_anos_ou_mais_de_idade.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/educacao_pe/csv/pessoas_de_10_anos_ou_mais_de_idade_grupos_de_anos_de_estudo.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/educacao_pe/csv/pessoas_que_frequentavam_creche_ou_escola.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/trabalho_rendimento_pe/csv/pessoas_de_10_anos_ou_mais_de_idade.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/trabalho_rendimento_pe/csv/pessoas_de_10_anos_ou_mais_de_idade_empregadas_no_trabalho_principal_da_semana_de_referencia.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/trabalho_rendimento_pe/csv/pessoas_de_10_anos_ou_mais_de_idade_ocupadas_na_semana_de_referência.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/trabalho_rendimento_pe/csv/valor_do_rendimento_nominal_mediano_mensal_do_trabalho_principal_das_pessoas_de_10_anos_ou_mais_de_idade.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/domicilios_particulares_permanentes.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/domicilios_particulares_permanentes_por_situacao_destino_do_lixo.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/domicilios_particulares_permanentes_por_situacao_tipo_abastecimento_agua.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/domicilios_particulares_permanentes_por_situacao_tipo_esgotamento_sanitario.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/media_de_anos_de_estudo_da_populacao_de_10_anos_ou_mais_de_idade.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/numero_medio_moradores_domicilios_particulares_permanentes_classe_de_salario.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/pessoas_de_60_ou_mais_anos_de_idade_aposentados_ou_nao.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/pessoas_de_60_ou_mais_anos_de_idade_por_classe_de_contribuicao.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/populacao_pe.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/populacao_residente_pe.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/populacao_residente_por_cor_pe.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/propocao_de_pessoas_de_15_anos_ou_mais_de_idade_com_menos_de_4_anos_de_estudo.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/proporcao_de_domicilios_com_criancas_de_ate_14_anos_de_idade.csv")
data_stack.append(data)
#data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/proporcao_de_pessoas_de_10_anos_ou_mais_de_idade_ocupadas_na_semana_de_referência.csv")
#data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/proporcao_de_pessoas_de_10_anos_ou_mais_de_idade_ocupadas_na_semana_de_referência_por_contribuicao_instituto_previdencia.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/proporcao_de_pessoas_de_10_anos_ou_mais_de_idade_ocupadas_na_semana_de_referência_por_posicao_na_ocupacao.csv")
data_stack.append(data)
data = pd.read_csv("data/censo_2000_pe/indicadores_sociais_pe/csv/taxa_de_escolarizacao_por_idade.csv")
data_stack.append(data)

In [313]:
for x in range(0,23):
    data_stack[x] = retirar_acento(data_stack[x])

In [314]:
data_s = pd.concat(data_stack, axis=1)
#teste.head()

In [316]:
tabela = tabela.set_index('municipio')
#tabela.head()

In [317]:
tabela_final = pd.concat([tabela,data_s], axis=1).drop('codigo_unidade_geografica', 1)
tabela_final = tabela_final.reset_index()
tabela_final = tabela_final.rename(columns={"index": "municipio"})
tabela_final.head()

,municipio,codigo_un,microrregiao,pop,homicidio,taxa_homicidio,pessoas_de_10_anos_ou_mais_de_idade_total,pessoas_de_10_anos_ou_mais_de_idade_alfabetizadas,pessoas_de_10_anos_ou_mais_de_idade_taxa_de_alfabetizacao_(%),pessoas_de_10_anos_ou_mais_de_idade_10_a_14_anos_total,...,"pessoas_de_10_anos_ou_mais_de_idade,_ocupadas_na_semana_de_referencia_por_posicao_na_ocupacao_(%)_empregado_sem_carteira","pessoas_de_10_anos_ou_mais_de_idade,_ocupadas_na_semana_de_referencia_por_posicao_na_ocupacao_(%)_trabalhador_domestico_total","pessoas_de_10_anos_ou_mais_de_idade,_ocupadas_na_semana_de_referencia_por_posicao_na_ocupacao_(%)_trabalhador_domestico_com_carteira","pessoas_de_10_anos_ou_mais_de_idade,_ocupadas_na_semana_de_referencia_por_posicao_na_ocupacao_(%)_trabalhador_domestico_sem_carteira",taxa_de_escolarizacao_(%)_total,taxa_de_escolarizacao_(%)_por_grupos_de_idade_0_a_6_anos,taxa_de_escolarizacao_(%)_por_grupos_de_idade_7_a_14_anos,taxa_de_escolarizacao_(%)_por_grupos_de_idade_15_a_17_anos,taxa_de_escolarizacao_(%)_por_grupos_de_idade_18_a_24_anos,taxa_de_escolarizacao_(%)_por_grupos_de_idade_25_anos_ou_mais
0,Abreu e Lima,260005,Recife,83486,32.0,38.329780,72426.0,63709.0,87.964267,8979.0,...,17.256190,6.893336,1.778760,5.114576,32.530620,41.945898,95.584063,78.143800,33.812041,3.816416
1,Afogados da Ingazeira,260010,Vale do Pajeu,34524,8.0,23.172286,26208.0,19629.0,74.896978,3807.0,...,18.747716,5.147196,0.534028,4.613167,29.143026,25.394765,89.645511,68.337465,27.082396,2.221973
2,Afranio,260020,Petrolina,14227,1.0,7.028889,11848.0,8158.0,68.855503,1824.0,...,23.356101,3.427852,0.038902,3.388949,33.557669,21.179424,95.555202,75.170484,32.956231,5.897674
3,Agrestina,260030,Brejo,18713,8.0,42.751029,15728.0,9599.0,61.031282,2263.0,...,23.695550,4.252085,0.665384,3.586701,31.677327,40.297104,92.067825,66.086919,25.231703,3.157923
4,Agua Preta,260040,Mata Meridional,25353,21.0,82.830434,21824.0,14347.0,65.739553,3784.0,...,24.163987,5.626785,1.142118,4.484667,33.183783,14.363835,86.474907,67.870384,37.045502,6.286884


In [318]:
list(tabela_final)

['municipio',
 'codigo_un',
 'microrregiao',
 'pop',
 'homicidio',
 'taxa_homicidio',
 'pessoas_de_10_anos_ou_mais_de_idade_total',
 'pessoas_de_10_anos_ou_mais_de_idade_alfabetizadas',
 'pessoas_de_10_anos_ou_mais_de_idade_taxa_de_alfabetizacao_(%)',
 'pessoas_de_10_anos_ou_mais_de_idade_10_a_14_anos_total',
 'pessoas_de_10_anos_ou_mais_de_idade_10_a_14_anos_alfabetizadas',
 'pessoas_de_10_anos_ou_mais_de_idade_10_a_14_anos_taxa_de_alfabetizacao_(%)',
 'pessoas_de_10_anos_ou_mais_de_idade_15_a_19_anos_total',
 'pessoas_de_10_anos_ou_mais_de_idade_15_a_19_anos_alfabetizadas',
 'pessoas_de_10_anos_ou_mais_de_idade_15_a_19_anos_taxa_de_alfabetizacao_(%)',
 'pessoas_de_10_anos_ou_mais_de_idade_20_anos_ou_mais_total',
 'pessoas_de_10_anos_ou_mais_de_idade_20_anos_ou_mais_alfabetizadas',
 'pessoas_de_10_anos_ou_mais_de_idade_20_anos_ou_mais_taxa_de_alfabetizacao_(%)',
 'codigo',
 'pessoas_de_10_anos_ou_mais_de_idade_total',
 'pessoas_de_10_anos_ou_mais_de_idade_grupos_de_anos_de_estudo_sem_

In [319]:
tabela_final.to_csv('../2017-2-projeto-lbs4-eebls-ktcn/tabela_final.csv', index=False)